In [3]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

import torch
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.datasets import register_coco_instances
from detectron2.data import get_detection_dataset_dicts
from detectron2.data.detection_utils import read_image
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode

import numpy as np
import os
import json
import cv2
from pycocotools import mask

import json
from detectron2.data.detection_utils import read_image
from PIL import Image
import numpy

import cv2
from matplotlib import pyplot as plt
from detectron2.utils.visualizer import Visualizer, VisImage

from detectron2.config import configurable
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads

from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, fast_rcnn_inference
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
from detectron2.layers import ShapeSpec, batched_nms
from typing import Dict, List, Optional, Tuple
from detectron2.modeling.poolers import ROIPooler
from torch.nn import functional as F

from detectron2.layers import ShapeSpec, batched_nms, cat, cross_entropy, nonzero_tuple
from detectron2.utils.events import get_event_storage
from detectron2.evaluation.coco_evaluation import COCOEvaluator

from detectron2.engine.defaults import DefaultTrainer
import logging
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.layers import ShapeSpec
import torch

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class NonCocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name)

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128


In [ ]:
training_categ = ['chess', 'skiing', 'weightlifting', 'climbing', 'cricket', 'flying', 'hockey', 'soccer', 'volleyball', 'tennis', 'skateboarding', 'swimming', 'rowing', 'roller skating', 'horse racing', 'steeplechase', 'jogging', 'gymnastics', 'golf', 'diving', 'car racing', 'boxing', 'bowling', 'billiard', 'beach volleyball', 'basketball', 'baseball', 'jumping', 'running', 'acrobatics', 'airplane', 'glider', 'helicopter', 'hot-air_balloon', 'bicycle', 'camper', 'convertible', 'jeep', 'limousine', 'sedan', 'taxi', 'wagon', 'carriage', 'motorcycle', 'bus', 'minibus', 'tram', 'trolleybus', 'road sign', 'traffic police', 'zebra crossing', 'boat', 'ferry', 'gondola', 'motorboat', 'sailing vessel', 'ship', 'yacht', 'sleigh', 'rocket', 'spaceship', 'train', 'car transporter', 'dumper', 'garbage truck', 'lorry', 'pickup', 'tow truck', 'truck', 'van', 'bulldozer', 'digger', 'forklift', 'tractor', 'artist', 'sculptor', 'accordionist', 'piper', 'cellist', 'clarinetist', 'conductor', 'flute player', 'guitar player', 'opera singer', 'percussionist', 'piano player', 'rapper', 'saxophonist', 'singer', 'trombonist', 'trumpeter', 'violinist', 'ballet dancer', 'cameraman', 'clown', 'dancer', 'makeup artist', 'photographer', 'writer', 'figure skating', 'off road motorcycling', 'motorcycle racing', 'baby carriage', 'fire engine', 'fireman', 'police car', 'police helicopter', 'mounted police', 'policeman', 'wheelchair', 'fishing', 'hunting', 'tank', 'hang gliding', 'rhythmic gymnastics', 'horse sleigh', 'ambulance', 'dog sleigh', 'military helicopter', 'police boat', 'motorcycle police', 'soldier', 'double-decker', 'bicycle racing', 'handball', 'armoured personnel carrier', 'military truck', 'rickshaw', 'scooter', 'pole vaulting']

cfg = get_cfg()
cfg.merge_from_file('/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml')
cfg.MODEL.WEIGHTS = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x/137260431/model_final_a54504.pkl'

cfg.DATALOADER.NUM_WORKERS = 1
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1400)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.TEST.EVAL_PERIOD = 2000
cfg.MODEL.DEVICE='cuda:1'

for folder_name in training_categ:
    try:
        DatasetCatalog.remove(folder_name)
        MetadataCatalog.remove(folder_name)
        print('Update dataset')
    except:
        print('New dataset')
    #register_coco_instances(folder_name,{},'/host/comparison/'+folder_name+'/'+folder_name+'_train.json','/host/comparison/'+folder_name+'/data/')
    register_coco_instances(folder_name,{},'/host/app1/server/uploads/'+folder_name+'_gt.json','/host/app1/server/uploads/'+folder_name+'/')
    
    try:
        del trainer
    except:
        print("Starting...")
    
    ind = 0
    img = DatasetCatalog.get(folder_name)[ind]['file_name']
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #v = Visualizer(img,MetadataCatalog.get(folder_name))
    #v.draw_dataset_dict(DatasetCatalog.get(folder_name)[ind])
    print(folder_name)

    #plt.figure(figsize=(15, 15))
    #plt.imshow(v.get_output().get_image())
    #plt.show()
    
    cfg.DATASETS.TRAIN = (folder_name,)
    cfg.DATASETS.TEST = ()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(folder_name).thing_classes)
    
    torch.cuda.init()
        
    trainer = NonCocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    trainer.checkpointer.save(folder_name)

New dataset
Starting...



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



chess
[02/27 21:30:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
    

WARNING [02/27 21:30:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/27 21:30:35 d2.data.datasets.coco]: Loaded 216 images in COCO format from /host/app1/server/uploads/chess_gt.json
[02/27 21:30:35 d2.data.build]: Removed 0 images with no usable annotations. 216 images left.
[02/27 21:30:35 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |   category   | #instances   |
|:----------:|:-------------|:----------:|:-------------|:------------:|:-------------|
|    man     | 54           |   clock    | 81           | chess player | 300          |
| chessboard | 290          |  chessman  | 4690         | water bottle | 55           |
|            |              |            |              |              |              |
|   total    | 5470         |            |              |              |              |
[02/27 21:30:35 d2.data.dataset_mapper]:

model_final_a54504.pkl: 178MB [00:19, 8.97MB/s]                              
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due

...

[02/27 21:54:55 d2.utils.events]:  eta: 0:24:15  iter: 699  total_loss: 0.9663  loss_cls: 0.2194  loss_box_reg: 0.4003  loss_mask: 0.2501  loss_rpn_cls: 0.01599  loss_rpn_loc: 0.1056  time: 2.0508  data_time: 1.1346  lr: 0.001  max_mem: 0M
[02/27 21:55:35 d2.utils.events]:  eta: 0:23:50  iter: 719  total_loss: 0.9372  loss_cls: 0.2118  loss_box_reg: 0.3642  loss_mask: 0.2443  loss_rpn_cls: 0.01591  loss_rpn_loc: 0.1083  time: 2.0486  data_time: 1.3447  lr: 0.001  max_mem: 0M
[02/27 21:56:17 d2.utils.events]:  eta: 0:23:26  iter: 739  total_loss: 0.8645  loss_cls: 0.2017  loss_box_reg: 0.3484  loss_mask: 0.2313  loss_rpn_cls: 0.0108  loss_rpn_loc: 0.0995  time: 2.0498  data_time: 1.4212  lr: 0.001  max_mem: 0M
[02/27 21:56:56 d2.utils.events]:  eta: 0:22:49  iter: 759  total_loss: 0.9809  loss_cls: 0.1916  loss_box_reg: 0.3681  loss_mask: 0.2333  loss_rpn_cls: 0.01336  loss_rpn_loc: 0.1227  time: 2.0478  data_time: 1.3213  lr: 0.001  max_mem: 0M
[02/27 21:57:39 d2.utils.events]:  eta: 0

[02/27 22:17:14 d2.utils.events]:  eta: 0:04:25  iter: 1359  total_loss: 0.8588  loss_cls: 0.2122  loss_box_reg: 0.3205  loss_mask: 0.2431  loss_rpn_cls: 0.01446  loss_rpn_loc: 0.1149  time: 2.0398  data_time: 1.2105  lr: 5e-05  max_mem: 0M
[02/27 22:17:55 d2.utils.events]:  eta: 0:03:47  iter: 1379  total_loss: 0.8615  loss_cls: 0.1812  loss_box_reg: 0.3273  loss_mask: 0.2211  loss_rpn_cls: 0.01184  loss_rpn_loc: 0.09659  time: 2.0398  data_time: 1.3776  lr: 5e-05  max_mem: 0M
[02/27 22:18:37 d2.utils.events]:  eta: 0:03:09  iter: 1399  total_loss: 0.8883  loss_cls: 0.2013  loss_box_reg: 0.3185  loss_mask: 0.232  loss_rpn_cls: 0.008829  loss_rpn_loc: 0.1004  time: 2.0405  data_time: 1.4463  lr: 5e-05  max_mem: 0M
[02/27 22:19:16 d2.utils.events]:  eta: 0:02:32  iter: 1419  total_loss: 0.8321  loss_cls: 0.1778  loss_box_reg: 0.3112  loss_mask: 0.2261  loss_rpn_cls: 0.01445  loss_rpn_loc: 0.09143  time: 2.0391  data_time: 1.2728  lr: 2.5e-06  max_mem: 0M
[02/27 22:19:56 d2.utils.events]

WARNING [02/27 22:21:58 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/27 22:21:58 d2.data.datasets.coco]: Loaded 223 images in COCO format from /host/app1/server/uploads/skiing_gt.json
[02/27 22:21:58 d2.data.build]: Removed 0 images with no usable annotations. 223 images left.
[02/27 22:21:58 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category   | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:-----------:|:-------------|
|     skier     | 322          | ski jacket  | 107          |  ski boot   | 346          |
|   ski pole    | 528          | ski helmet  | 148          |     ski     | 514          |
|  race number  | 108          | ski trouser | 116          | ski googles | 226          |
| ski racing .. | 157          |             |              |             |              |
|     total     | 25

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (10, 256, 1, 

[02/27 22:21:58 d2.engine.train_loop]: Starting training from iteration 0
[02/27 22:22:48 d2.utils.events]:  eta: 0:47:02  iter: 19  total_loss: 2.928  loss_cls: 1.136  loss_box_reg: 0.9127  loss_mask: 0.61  loss_rpn_cls: 0.1094  loss_rpn_loc: 0.217  time: 2.2364  data_time: 1.8484  lr: 0.001  max_mem: 0M
[02/27 22:23:39 d2.utils.events]:  eta: 0:48:33  iter: 39  total_loss: 2.43  loss_cls: 0.849  loss_box_reg: 0.8696  loss_mask: 0.4845  loss_rpn_cls: 0.06038  loss_rpn_loc: 0.1412  time: 2.3957  data_time: 1.8987  lr: 0.001  max_mem: 0M
[02/27 22:24:20 d2.utils.events]:  eta: 0:46:50  iter: 59  total_loss: 2.165  loss_cls: 0.7489  loss_box_reg: 0.8291  loss_mask: 0.4191  loss_rpn_cls: 0.05734  loss_rpn_loc: 0.1606  time: 2.2746  data_time: 1.3813  lr: 0.001  max_mem: 0M
[02/27 22:25:11 d2.utils.events]:  eta: 0:48:54  iter: 79  total_loss: 1.966  loss_cls: 0.6084  loss_box_reg: 0.8136  loss_mask: 0.3244  loss_rpn_cls: 0.03985  loss_rpn_loc: 0.1231  time: 2.3529  data_time: 1.9179  lr: 